In [2]:
# インポート
import qutip as qt
import numpy as np
from pathlib import Path
from datetime import datetime
import matplotlib as mpl
from matplotlib.ticker import MultipleLocator
import matplotlib.patheffects as pe
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from functools import lru_cache
import requests

# グラフ設定
mpl.rcParams.update(mpl.rcParamsDefault)
plt.style.use('default')
mpl.rcParams.update({
    "axes.labelsize": 18,   # 軸ラベル
    "xtick.labelsize": 16,  # 目盛り数字
    "ytick.labelsize": 16,
    "lines.linewidth": 2.2, # 線の太さ
    "axes.titlesize": 16,
})
def inward_ticks(ax, major_len=7, minor_len=4, width=1.2):
    ax.minorticks_on()
    ax.tick_params(which="both", direction="in", top=True, right=True,
                   length=major_len, width=width)
    ax.tick_params(which="minor", length=minor_len, width=width)

# パラメータ
Lx, Ly = 4, 4
cut_pos = Lx // 2 - 1
N = Lx * Ly
idx = lambda x, y: y * Lx + x
J_list = [0.5, 1.0, 2.0]
Jstamp = '-'.join(map(str, J_list))
Tcut_list = np.linspace(0.0, 5.0, 11)

# データ読み込み
# fname = DATADIR / "SA_vs_Tcut_N{N}_J{Jstamp}_.npz" 

# dat = np.load(fname)

# J_list = dat["J_list"]
# Tcut_list = dat["Tcut_list"]
# EE      = dat["EE"]

#データ保存
DATADIR = Path("../../data/tfim")


# 通知設定
DISCORD_WEBHOOK_URL = "https://discord.com/api/webhooks/1439055708988375082/6iiXA8J_3Bn1RPWZ8XmPfmfSJp_PkGdaIzt495Ao2fbu_a09VnMJpxmrnSyfN4Wtyc7T"
def notify_discord(message: str):
    payload = {"content": message}
    try:
        r = requests.post(DISCORD_WEBHOOK_URL, json=payload, timeout=5)
        r.raise_for_status()
        print("✅ Discord 通知送信 OK")
    except Exception as e:
        print("⚠ Discord 通知に失敗:", e)

In [ ]:
# ========= キャッシュ付きヘルパ =========

@lru_cache(maxsize=None)
def get_ops(N):
    """Nだけで決まる演算子たち"""
    cut_pos = N // 2 - 1
    sx, sy, sz = [0.5 * M for M in (qt.sigmax(), qt.sigmay(), qt.sigmaz())]
    I2 = qt.qeye(2)

    

    def op_at(i, op):
        return qt.tensor([op if k == i else I2 for k in range(N)])

    Sx = [op_at(i, sx) for i in range(N)]
    Sy = [op_at(i, sy) for i in range(N)]
    Sz = [op_at(i, sz) for i in range(N)]

# =======================ここまで
    
    def H_exchange(i):
        return Sz[i] * Sz[i+1]

    Hcut_unit   = H_exchange(cut_pos)
    Hex_rest_unit = sum(H_exchange(i) for i in range(N-1) if i != cut_pos)
    Hhx_unit    = sum(Sx[i] for i in range(N))

    left_sites = list(range(N//2))
    return Hcut_unit, Hex_rest_unit, Hhx_unit, left_sites


@lru_cache(maxsize=None)
def get_psi0(N, J, hx):
    """(N, J, hx)で決まる初期基底状態"""
    Hcut_unit, Hex_rest_unit, Hhx_unit, _ = get_ops(N)
    H_full = -J * (Hcut_unit + Hex_rest_unit) - hx * Hhx_unit
    _, psi0 = H_full.groundstate(sparse=True)
    return psi0

# シミュレーション関数
# ========= メイン: Tcut ごとの時間発展 =========


def simulate(N, J, hx, Tcut):
    Hcut_unit, Hex_rest_unit, Hhx_unit, left_sites = get_ops(N)
    psi0 = get_psi0(N, J, hx)

    # H(t) = H_static + (-J * Hcut_unit) * ramp(t)
    H_static = -J * Hex_rest_unit - hx * Hhx_unit

    def ramp(t, T_cut):
        if T_cut == 0.0:
            # 瞬時切断：t<=0で1, t>0で0
            return 1.0 if t <= 0.0 else 0.0
        if t <= 0.0:
            return 1.0
        if t >= T_cut:
            return 0.0
        return 1.0 - t / T_cut

    Ht = [H_static, [-J * Hcut_unit, lambda t, args: ramp(t, Tcut)]]

    # ---- Tcut=0 は積分せず即時評価 ----
    if Tcut <= 0.0:
        rhoL = qt.ptrace(psi0, left_sites)
        return float(qt.entropy_vn(rhoL, base=2))

    # ---- 長時間を区間分割して 2点出力でつないでいく ----
    # 例: 1000刻み（Tcut=10000なら10区間）
    seg_len = 1000.0
    nseg = max(1, int(np.ceil(Tcut / seg_len)))
    grid = np.linspace(0.0, Tcut, nseg + 1)

    # 剛性が強い/臨界近傍を想定して bdf、必要なら adams に変更
    opts = qt.Options(
        method='bdf',          # 剛性に強い。速さ優先なら 'adams'
        rtol=1e-6,
        atol=1e-8,
        nsteps=200_000,        # 内部最大ステップ数の上限を増やす
        max_step=(Tcut/nseg)/200.0,  # 各区間長/200 を目安（安定しないなら小さく）
        store_states=True,     # 2点出力なのでメモリ負担は僅少
        progress_bar=None,
    )

    psi = psi0
    for k in range(nseg):
        t0, t1 = float(grid[k]), float(grid[k+1])
        # 各区間は tlist=[t0, t1] の2点出力（終端だけ使う）
        res = qt.sesolve(Ht, psi, [t0, t1], e_ops=[], options=opts)
        psi = res.states[-1]

    # 終端で部分トレース→エントロピー
    rhoL = qt.ptrace(psi, left_sites)
    return float(qt.entropy_vn(rhoL, base=2))
